In [1]:
%load_ext autoreload 
%autoreload 2

In [5]:
from preprocessing import ner_training, expansion, seed_data_extraction
from postprocessing import training_data_generation, extract_new_entities, filtering
import config as cfg
import gensim
import elasticsearch
from config import ROOTPATH 

In [50]:
es = elasticsearch.Elasticsearch([{'host': 'localhost', 'port': 9200}])
model_doc2vec = gensim.models.Doc2Vec.load(cfg.ROOTPATH + '/models/doc2vec.model')

In [81]:
seeds = ['clueweb', 'imagenet', 'flickr', 'webkb', 'netflix', 'imdb']
context_words = ['dataset', 'corpus', 'collection', 'repository', 'benchmark', 'website']
sentence_expansion = True
training_cycles = 2
model_name = 'dataset'
training_cycle = 0

In [82]:
for cycle in range(training_cycles):
    print(cycle)

0
1


#  seed_data_extraction.sentence_extraction(seeds)

In [8]:
seed_data_extraction.sentence_extraction(model_name, seeds)

Started training data extraction
Selected seed terms ['clueweb', 'webkb', 'imagenet', 'netflix', 'flickr', 'imdb']
clueweb
Got 393 hits in ES
webkb
Got 124 hits in ES
imagenet
Got 41 hits in ES
netflix
Got 548 hits in ES
flickr
Got 2521 hits in ES
imdb
Got 448 hits in ES
2546 sentences added for training in iteration

Process finished with 6 seeds


#  expansion.term_expansion_proteins

In [10]:
expansion.term_expansion(model_name, 0)

Started to extract generic named entity from sentences...
Finished processing sentences with 5259 new possible entities
started clustering
......added 1548 expanded terms
..........................................

In [12]:
expansion.sentence_expansion(model_name, 0, model_doc2vec)

Expanding sentences
Added 2525 expanded sentences to the 4101 original


In [15]:
type(model_doc2vec)

gensim.models.doc2vec.Doc2Vec

# trainingdata_generation.generate_training_term_expansion_only

In [43]:
trainingdata_generation.generate_training_term_expansion_only(model_name, 0)

Labelling
4095 lines labelled


# trainingdata_generation.label_sentences_term_sentence_expansion

In [22]:
trainingdata_generation.sentence_labelling(model_name, 0, False)

Labelling
4101 lines labelled


# ner_training.create_austenprop

In [72]:
ner_training.create_prop(model_name, 0, sentence_expansion)

In [73]:
ner_training.train_model(model_name, 0)

In [79]:
extract_new_entities.ne_extraction(model_name, 0, sentence_expansion)

started extraction..... dataset cycle 0
815
...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................797
.........................................................................................................................................

In [87]:
filtering.pmi(model_name, training_cycle)

filtering...
Started  pmi filtering...
Calculating npd with dataset
!....!.....!....!.!.........!......!..!.....!....................................!...!...........!.....!........!....!......!...!.!................!.............................!.............!...!....!...!..!................!.....!..!...!........!......!..!.!.............!.....!..!............!..!......!...!.....!.....................!....!......!...........!.......!........!......!..............................!..!..!.....!.....!.....!..!..!...!..!..!.!....!.......!........!...!.......!.!.!.......!...!..!........!....!.......!.......!.!..!......!.....!...!.!.....!....!..!...............!........................!.....!.....!.........!.!..........!.....!........!.!....!...............!....!........!......!....!.!.!.!..!................!.!.....!.......!..............!................!................!..............!.!.!....!.....!.!.....!........!..!...!...!.!...........!......!..........!..!.....!....!.........!........

In [19]:
import codecs
import numpy
import string
from numbers import Number
from xml.etree import ElementTree

import nltk
import requests
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

from config import ROOTPATH
from postprocessing import normalized_pub_distance

stopword_path = ROOTPATH + "/data/stopword_en.txt"
stopword_list = []
with open(stopword_path, 'r') as file:
        for sw in file.readlines():
            stopword_list.append(sw.strip())

In [23]:
extracted_entities = []
path = ROOTPATH + '/processing_files/' + model_name + '_extracted_entities_' + str(training_cycle) + '.txt'
with open(path, "r") as f:
    for e in f.readlines():
        extracted_entities.append(e.strip())
print('Filtering', len(extracted_entities), 'with WordNet and Stopwords')

Filtering 1854 with WordNet and Stopwords


In [29]:
extracted_entities = []
path = ROOTPATH + '/processing_files/' + model_name + '_extracted_entities_' + str(training_cycle) + '.txt'
with open(path, "r") as f:
    extracted_entities = [e.strip() for e in f.readlines()]
print('Filtering', len(extracted_entities), 'with WordNet and Stopwords')

Filtering 1854 with WordNet and Stopwords


In [24]:
ws_filtered = []
stopword_filtered = [word for word in set(extracted_entities) if word.lower() not in stopwords.words('english')]
print(len(stopword_filtered))
stopword_filtered = [word for word in set(stopword_filtered) if word.lower() not in stopword_list]
print(len(stopword_filtered))

1846
1801


In [25]:
filter_by_wordnet = [word for word in stopword_filtered if not wordnet.synsets(word)]

In [27]:
filter_by_wordnet = []
for word in set(stopword_filtered):
    if not wordnet.synsets(word):
        filter_by_wordnet.append(word)

In [26]:
print(len(filter_by_wordnet))

1055


In [28]:
print(len(filter_by_wordnet))

1055


In [36]:
path = ROOTPATH + '/processing_files/' + model_name + '_extracted_entities_' + str(training_cycle) + '.txt'
with open(path, "r") as f:
    extracted_entities = [e.strip().lower() for e in f.readlines()]

In [44]:
filtering.filter_st_pmi_kbl_ec(model_name, training_cycle, seeds)

Filtering 1854 entities with embedded clustering
........716 entities are kept from the total of 1854


In [67]:
filtering.filter_pmi(model_name, training_cycle, context_words)

Filtering 1854 entities with PMI
678 entities are kept from the total of 1854


In [77]:
filtering.majority_vote(model_name, training_cycle)

Filtering 1854 entities by vote of selected filter methods
598 entities are kept from the total of 1854
